# Tabular playground May

In [24]:
%pylab inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import pandas as pd
import numpy as np
from lib import *
from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC

Populating the interactive namespace from numpy and matplotlib


C:\Users\User\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
class Judge:
    
    def __init__(self):
        self.params = {}
        return None
    
    def set_algorithms(self, algorithms):
        self.algorithms = algorithms
        return self
    
    def set_data(self, X, y):
        self.X = X
        self.y = y
        return self
    
    def set_metrics(self, metrics):
        self.metrics = metrics
        return self
    
    def set_params(self, params):
        self.params = params
        return self
    
    def get_performance(self, metric, algorithm, grid):
        best_algorithm = algorithm
        if bool(grid):
            best_algorithm = GridSearchCV(estimator = algorithm, param_grid = grid)  # inner loop
        scores = cross_validate(best_algorithm, X = self.X, y = self.y, scoring = metric, cv = 10)  # outer_loop
        score = np.mean(scores['test_score'])
        score = round(score, 2)
        
        return score
        
    def get_table(self):
        metrics_results = {}
        for metric in self.metrics:
            algorithms_results = {}
            for label, algorithm in self.algorithms.items():
                grid = {}
                if label in self.params.keys():
                    grid = self.params[label]
                algorithms_results[label] = self.get_performance(metric, algorithm, grid)
            metrics_results[metric] = algorithms_results
            
        df = pd.DataFrame.from_dict(metrics_results)
                
        return df
    

In [3]:
df_train = pd.read_csv("C:/Users/User/Downloads/playground_maggio_train.csv/train.csv")
df_test = pd.read_csv("C:/Users/User/Downloads/playground_maggio_test.csv/test.csv")

In [4]:
df_test.describe() #Che aspetto hanno i dati

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_28,f_29,f_30
count,7.000000e+05,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,...,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000
mean,1.250000e+06,-0.000087,-0.000732,-0.000385,0.000705,0.000800,-0.000468,0.002550,2.030819,2.056923,...,-0.173972,-0.153938,-0.005623,-0.371983,-0.340683,0.174245,0.356288,0.604213,0.346090,1.003081
std,2.020727e+05,1.000264,0.997131,1.000317,1.001758,0.999088,1.000249,0.999536,1.655909,1.591726,...,2.397938,2.484198,2.450907,2.452619,2.388938,2.417555,2.477713,238.888993,0.475723,0.819673
min,9.000000e+05,-4.658018,-4.922718,-4.457561,-4.567419,-4.675301,-5.141356,-4.782164,0.000000,0.000000,...,-11.089910,-12.186778,-11.700814,-12.104478,-11.838417,-13.312784,-13.462486,-1204.243716,0.000000,0.000000
25%,1.075000e+06,-0.675708,-0.674040,-0.675961,-0.674561,-0.674696,-0.674444,-0.671481,1.000000,1.000000,...,-1.794757,-1.816739,-1.641035,-2.021342,-1.948923,-1.443815,-1.263991,-158.660917,0.000000,0.000000
50%,1.250000e+06,-0.000395,-0.000241,-0.000723,-0.000294,0.000276,-0.001674,0.002623,2.000000,2.000000,...,-0.184945,-0.149100,0.033726,-0.396281,-0.339431,0.165229,0.399981,0.671386,0.000000,1.000000
75%,1.424999e+06,0.675788,0.671415,0.673105,0.676376,0.674542,0.674146,0.676590,3.000000,3.000000,...,1.447103,1.508052,1.664146,1.251361,1.269879,1.794429,2.025163,159.789006,1.000000,2.000000
max,1.599999e+06,5.761950,4.768073,4.599902,4.899904,4.494312,4.701004,4.673145,16.000000,13.000000,...,10.691366,11.999494,10.991597,11.366652,10.236546,11.294340,12.811976,1141.957328,1.000000,2.000000


In [5]:
prediction=pd.DataFrame(df_test.id)
df_train.drop(['id'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)
#La colonna "id" non ha nessuna rilevanza

In [6]:
df_train.target.value_counts() #I dataset sono bilanciati

0    462161
1    437839
Name: target, dtype: int64

In [7]:
print(df_train.isnull().sum())
print(df_test.isnull().sum()) #Non ci sono valori nulli

f_00      0
f_01      0
f_02      0
f_03      0
f_04      0
f_05      0
f_06      0
f_07      0
f_08      0
f_09      0
f_10      0
f_11      0
f_12      0
f_13      0
f_14      0
f_15      0
f_16      0
f_17      0
f_18      0
f_19      0
f_20      0
f_21      0
f_22      0
f_23      0
f_24      0
f_25      0
f_26      0
f_27      0
f_28      0
f_29      0
f_30      0
target    0
dtype: int64
f_00    0
f_01    0
f_02    0
f_03    0
f_04    0
f_05    0
f_06    0
f_07    0
f_08    0
f_09    0
f_10    0
f_11    0
f_12    0
f_13    0
f_14    0
f_15    0
f_16    0
f_17    0
f_18    0
f_19    0
f_20    0
f_21    0
f_22    0
f_23    0
f_24    0
f_25    0
f_26    0
f_27    0
f_28    0
f_29    0
f_30    0
dtype: int64


# Visualizzazione delle features

In [8]:
continuous = df_train.select_dtypes(include = 'float64')
integers = df_train.select_dtypes(include = 'int64').drop(columns = ['target'])
text = df_train.select_dtypes(include = 'object')
numeric = df_train.select_dtypes(exclude = 'object').drop(columns = ['target'])

target = df_train.target

In [ ]:
for column in continuous.columns:
    sns.displot(data = continuous, x = column)
    plt.show()

In [ ]:
for column in integers.columns:
    sns.countplot(x = column, data = integers)
    plt.show()

In [ ]:
for column in integers.columns:
    sns.barplot(integers[column], df_train.target)
    plt.show()

Dalla visualizzazione delle feature continue e intere è possibile fare due considerazioni:

1) Non sono presenti outliers rilevanti. Nelle distribuzioni a numeri interi non ve ne sono affatto, nelle distribuzioni continue (tutte Gaussiane) ve ne saranno, ma saranno semplicemente degli outliers statisticamente naturali, che possono arricchire  modello predittivo.

2) I numeri interi sono distribuiti tutti (ad eccezione delle features f_29 e f_30) con delle simil distribuzioni gamma. Dunque probabilmente non sono features categoriche, bensì numeriche.

In [ ]:
text

In [9]:
def DIVIDER_f_27(df):
    for i in range(10):
        lista=[]
        for parola in df.f_27.values:
            lista.append(parola[i])
        df["f_27_" + str(i)] = lista
    df.drop(["f_27"], axis=1, inplace=True)
    return

In [10]:
DIVIDER_f_27(text)

C:\Users\User\AppData\Local\Temp/ipykernel_14080/196705554.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["f_27_" + str(i)] = lista
C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [11]:
DIVIDER_f_27(df_test)
DIVIDER_f_27(df_train)

In [12]:
print(text.f_27_0.unique())
print(text.f_27_1.unique())
print(text.f_27_2.unique())
print(text.f_27_3.unique())
print(text.f_27_4.unique())
print(text.f_27_5.unique())
print(text.f_27_6.unique())
print(text.f_27_7.unique())
print(text.f_27_8.unique())
print(text.f_27_9.unique())

['A' 'B']
['B' 'C' 'A' 'D' 'E' 'G' 'F' 'H' 'I' 'J' 'K' 'L' 'M' 'N']
['A' 'B']
['B' 'C' 'E' 'D' 'H' 'I' 'G' 'A' 'F' 'J' 'K' 'L' 'M' 'N' 'O']
['D' 'C' 'A' 'B' 'F' 'E' 'G' 'I' 'H' 'J' 'K' 'L' 'M' 'O']
['A' 'B']
['D' 'C' 'A' 'E' 'B' 'G' 'H' 'F' 'I' 'J' 'M' 'K' 'L' 'N' 'O']
['B' 'C' 'K' 'H' 'E' 'P' 'D' 'A' 'S' 'T' 'J' 'Q' 'N' 'R' 'G' 'M' 'F' 'O'
 'I' 'L']
['A' 'E' 'C' 'F' 'D' 'H' 'B' 'G' 'I' 'J' 'L' 'K' 'M' 'N' 'O']
['B' 'D' 'E' 'A' 'C' 'K' 'F' 'G' 'H' 'J' 'I' 'L' 'M' 'N' 'O']


In [13]:
print(df_test.f_27_0.unique())
print(df_test.f_27_1.unique())
print(df_test.f_27_2.unique())
print(df_test.f_27_3.unique())
print(df_test.f_27_4.unique())
print(df_test.f_27_5.unique())
print(df_test.f_27_6.unique())
print(df_test.f_27_7.unique())
print(df_test.f_27_8.unique())
print(df_test.f_27_9.unique())

['B' 'A']
['A' 'F' 'B' 'E' 'C' 'D' 'I' 'H' 'G' 'K' 'J' 'L' 'N' 'M' 'O']
['A' 'B']
['A' 'B' 'C' 'E' 'D' 'H' 'G' 'F' 'L' 'J' 'I' 'M' 'K' 'N' 'O']
['B' 'A' 'C' 'F' 'D' 'E' 'H' 'G' 'L' 'I' 'J' 'K' 'M' 'N']
['A' 'B']
['D' 'E' 'B' 'C' 'F' 'A' 'H' 'G' 'I' 'J' 'L' 'K' 'M' 'N' 'O']
['L' 'G' 'K' 'Q' 'A' 'O' 'T' 'R' 'B' 'I' 'D' 'J' 'E' 'H' 'C' 'P' 'M' 'S'
 'F' 'N']
['A' 'C' 'E' 'B' 'D' 'G' 'F' 'I' 'J' 'N' 'H' 'L' 'K' 'M']
['C' 'B' 'E' 'F' 'A' 'G' 'D' 'H' 'I' 'L' 'J' 'K' 'M' 'N' 'O']


Ci sono lettere dalla A alla T, quindi costruisco un dizionario

In [14]:
letters_dict={"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5 , "G": 6, "H":  7, "I": 8, "J": 9, "K": 10, "L": 11, "M": 12, "N": 13, "O": 14, "P": 15, "Q": 16, "R": 17, "S": 18, "T": 19}


In [ ]:
for column in text.columns:
    sns.countplot(x = column, data = text, order=list(letters_dict.keys()))
    plt.show()


In [ ]:
for column in text.columns:
    sns.barplot(text[column], df_train.target, order=list(letters_dict.keys()))
    plt.show()

Avendo completà libertà di scegliere se le variabili sono categoriche o no, guardando alle tendenze sul target considero una certa ordinalità sull'ordine delle lettere, ad eccezione delle features f_27_0, f_27_2 e f_27_5 che sono binarie.

In [15]:
non_categoriche= list(continuous.columns)

In [16]:
non_categoriche.append("f_07")
non_categoriche.append("f_08")
non_categoriche.append("f_09")
non_categoriche.append("f_10")
non_categoriche.append("f_11")
non_categoriche.append("f_12")
non_categoriche.append("f_13")
non_categoriche.append("f_14")
non_categoriche.append("f_15")
non_categoriche.append("f_16")
non_categoriche.append("f_17")
non_categoriche.append("f_18")
non_categoriche.append("f_27_1")
non_categoriche.append("f_27_3")
non_categoriche.append("f_27_4")
non_categoriche.append("f_27_6")
non_categoriche.append("f_27_7")
non_categoriche.append("f_27_8")
non_categoriche.append("f_27_9")

In [ ]:
non_categoriche

In [17]:
def OHE_f_27(df):
    for i in range(10):
        d = pd.get_dummies(df["f_27_" + str(i)], prefix = "f_27_" + str(i))
        df = pd.concat([df, d], axis = 1)
        df = df.drop(["f_27_" + str(i)], axis = 1)
    return df

def ENCODER_f_27(df):
    letters_dict={"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5 , "G": 6, "H":  7, "I": 8, "J": 9, "K": 10, "L": 11, "M": 12, "N": 13, "O": 14, "P": 15, "Q": 16, "R": 17, "S": 18, "T": 19}
    df=df.replace(letters_dict)
    return df

In [18]:
df_train=ENCODER_f_27(df_train)
df_test=ENCODER_f_27(df_test)

In [19]:
df_train.head()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,f_27_0,f_27_1,f_27_2,f_27_3,f_27_4,f_27_5,f_27_6,f_27_7,f_27_8,f_27_9
0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,1,...,0,1,0,1,3,0,3,1,0,1
1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,4,...,0,2,0,2,2,0,3,2,4,1
2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,2,...,0,0,0,4,0,1,2,10,0,3
3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,1,...,1,3,1,1,0,0,2,1,2,1
4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,0,...,1,3,1,2,1,1,2,7,5,4


In [20]:
df_train.drop(['target'], axis=1, inplace=True)

In [ ]:
target

In [21]:
x_prova=df_train.copy()

In [22]:
y_prova=target

In [25]:
preprocessing_step = [
    ('ct', ColumnTransformer(
        [('minmax', MinMaxScaler(),non_categoriche)])),
    ('fs', SelectKBest(chi2))
]

preprocessing_step_2 = [
    ('minmax', MinMaxScaler()),
    ('fs', SelectKBest(chi2))
]

pipe_dt_01 = Pipeline(preprocessing_step + [('dt', DecisionTreeClassifier())], verbose = False)
pipe_dt_02 = Pipeline(preprocessing_step_2 + [('dt', DecisionTreeClassifier())], verbose = False)
pipe_dt_03 = Pipeline([('dt', DecisionTreeClassifier())], verbose = False)
#pipe_rf_01 = Pipeline(preprocessing_step + [('rf', RandomForestClassifier())], verbose = False)
#pipe_rf_02 = Pipeline(preprocessing_step_2 + [('rf', RandomForestClassifier())], verbose = False)
#pipe_rf_03 = Pipeline([('rf', RandomForestClassifier())], verbose = False)
pipe_gnb = Pipeline([('gnb', GaussianNB())], verbose = False)

pipelines = {
    'pipe_dt_01': pipe_dt_01,
    'pipe_dt_02': pipe_dt_02,
    'pipe_dt_03': pipe_dt_03,
    #'pipe_rf_01': pipe_rf_01,
    #'pipe_rf_02': pipe_rf_02,
    #'pipe_rf_03': pipe_rf_03,
    'pipe_gnb': pipe_gnb
    
}

# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
metrics = [
    'accuracy',
    'roc_auc',
    'precision',
    'recall'
]

params = {
    'pipe_dt_01': {
        'dt__min_samples_leaf': [1],
        'dt__max_depth': [10, 15, 20],
        'fs__k': [5,10,15]
    },
    'pipe_rf_01': {
        'fs__k': [5,10,15],
        'rf__n_estimators': [100, 200],
        'rf__max_depth': [ 5, 10, 15]
    },
    'pipe_gnb': {},
    
    'pipe_dt_02': {
        'dt__min_samples_leaf': [1],
        'dt__max_depth': [10, 15, 20],
        'fs__k': [5,10,15]
    },
    'pipe_rf_02': {
        'fs__k': [5,10,15],
        'rf__n_estimators': [100, 200],
        'rf__max_depth': [ 5, 10, 15]
    },
    'pipe_dt_03': {
        'dt__min_samples_leaf': [1],
        'dt__max_depth': [10, 15, 20],
    },
    'pipe_rf_03': {
        'rf__n_estimators': [100, 200],
        'rf__max_depth': [ 5, 10, 15]
    }
        
}
judge = Judge()
judge.set_data(x_prova, y_prova)
judge.set_algorithms(pipelines)
judge.set_metrics(metrics)
judge.set_params(params)
judge.get_table()

,accuracy,roc_auc,precision,recall
pipe_dt_01,0.65,0.70,0.65,0.59
pipe_dt_02,0.73,0.78,0.73,0.69
pipe_dt_03,0.77,0.82,0.78,0.73
pipe_gnb,0.68,0.74,0.69,0.63


In [26]:
best_pipe = pipe_dt_03
grid = params['pipe_dt_03']

clf = GridSearchCV(best_pipe, param_grid = grid)
clf.fit(x_prova, y_prova)
clf.best_estimator_

Pipeline(steps=[('dt', DecisionTreeClassifier(max_depth=15))])

In [ ]:
prediction

In [ ]:
clf.predict_proba(df_test)[:,1]

In [ ]:
prediction['target']=clf.predict_proba(df_test)[:,1]

In [ ]:
prediction.to_csv("C:/Users/User/Desktop/predizioni_playground_maggio_2022.csv", index=False)